# Get a summary of training status of all alive subjects in a lab

In [1]:
import datajoint as dj
from ibl_pipeline import subject, action, acquisition, behavior
from ibl_pipeline.analyses import behavior as behavior_analyses
import re

Connecting shan@datajoint-rds.cyuksi65nrdq.us-east-1.rds.amazonaws.com:3306


## Get all alive subjects in a lab with water administration and weight

In [2]:
subjects = (subject.Subject - subject.Death) & 'lab_name = "churchlandlab"' \
    & 'sex!="U"' & action.Weighing & action.WaterAdministration

In [3]:
subjects

lab_name name of lab,subject_nickname nickname,subject_uuid,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_source name of source,responsible_user username,protocol_number protocol number,subject_description


## List subjects with some training summaries
>* current training protocol
>* date of the last session
>* current training status

First, let's find the last session for each subject, including the protocol used in the last session.

In [6]:
subjects = (subject.Subject - subject.Death) & 'lab_name = "churchlandlab"' \
    & 'sex!="U"'

In [7]:
last_sessions = subjects.aggr(acquisition.Session & 'task_protocol!="NULL"', 
                              'task_protocol', session_start_time='max(session_start_time)')
last_sessions

lab_name name of lab,subject_nickname nickname,task_protocol,session_start_time calculated attribute
churchlandlab,CSHL_001,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-03-19 13:46:57
churchlandlab,CSHL_002,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-03-19 15:02:34
churchlandlab,CSHL_003,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-03-19 16:04:20
churchlandlab,CSHL_004,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-03-19 16:45:15
churchlandlab,CSHL_005,_iblrig_tasks_habituationChoiceWorld3.7.3,2019-03-19 17:56:10
churchlandlab,CSHL_006,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-03-19 13:47:29
churchlandlab,CSHL_007,_iblrig_tasks_habituationChoiceWorld3.7.4,2019-03-19 14:41:50


Second, get the current training status

In [8]:
last_sessions * behavior_analyses.SessionTrainingStatus

lab_name name of lab,subject_nickname nickname,session_start_time start time,task_protocol,training_status
churchlandlab,CSHL_001,2019-03-19 13:46:57,_iblrig_tasks_habituationChoiceWorld3.7.3,training in progress
churchlandlab,CSHL_002,2019-03-19 15:02:34,_iblrig_tasks_habituationChoiceWorld3.7.3,training in progress
churchlandlab,CSHL_003,2019-03-19 16:04:20,_iblrig_tasks_habituationChoiceWorld3.7.3,training in progress
churchlandlab,CSHL_004,2019-03-19 16:45:15,_iblrig_tasks_habituationChoiceWorld3.7.3,training in progress
churchlandlab,CSHL_005,2019-03-19 17:56:10,_iblrig_tasks_habituationChoiceWorld3.7.3,trained
churchlandlab,CSHL_006,2019-03-19 13:47:29,_iblrig_tasks_habituationChoiceWorld3.7.4,training in progress
churchlandlab,CSHL_007,2019-03-19 14:41:50,_iblrig_tasks_habituationChoiceWorld3.7.4,trained
